In [1]:
### lets import all the necessary packages !
import pandas as pd
import numpy as np

import os
import time

import json
import string

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import iplot

# API
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# NLP and Sentiment Analysis
import nltk
import emoji
from textblob import TextBlob
#from collections import Counter
#from wordcloud import WordCloud , STOPWORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

# GPU
from multiprocessing import Pool
import cupy as cp
import cudf as cd
import torch
#from nltk.sentiment import SentimentIntensityAnalyzer
#import dask.dataframe as dd





In [ ]:
# Read CSV file into a DataFrame
# df = pd.read_csv('video_comments.csv')

# # Save DataFrame as Parquet file
# df.to_parquet('video_comments.parquet')

In [9]:
# df = pd.read_csv('video_details.csv')

# # Save DataFrame as Parquet file
# df.to_parquet('video_details.parquet')

In [2]:



%time pdf_df = pd.read_parquet('video_comments.parquet')

%time cd_parquet = cd.read_parquet('video_comments.parquet')





CPU times: user 3.84 s, sys: 799 ms, total: 4.64 s
Wall time: 4.25 s
CPU times: user 1.51 s, sys: 434 ms, total: 1.94 s
Wall time: 1.93 s


In [4]:
print('Dimensions:', len(df_parquet))

Dimensions: 7379384


In [6]:
df_parquet.head()

,video_id,comment_text,likes,replies
0,34Na4j8AVgA,Playing this song in a car in the middle of th...,41377,401
1,34Na4j8AVgA,This song is never gonna die in my heart 💜,14,0
2,34Na4j8AVgA,6 years have passed and this song is still one...,130,0
3,34Na4j8AVgA,لن امل من سماع هذه التحفة الفنية تستحق 10 مليا...,1,0
4,34Na4j8AVgA,Still a hit after 6 years I miss the old weekend,0,0


CSV file read time: 1596.9153501987457 seconds
Parquet file read time: 1159.6301338672638 seconds


In [40]:
## Load the comments into a Dask DataFrame
#num_partitions = 4  # Example: Set the desired number of partitions
#df_dask = dd.from_pandas(df_parquet, npartitions=num_partitions)

In [41]:
## Explicitly specify the data types of columns if necessary
#df_dask['comment_text'] = df_dask['comment_text'].astype(str)

## Extract the comment_text column as a Dask Series
#comments = df_dask['comment_text']

[ 0.2023  0.9108  0.4215  0.6369 -0.1531  0.8519 -0.7351  0.0258  0.
  0.9711]
Elapsed time (Dask + cupy): 934.092068195343 seconds


In [ ]:
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# GPU
import cupy as cp
import cudf as cd



%time cd_parquet = cd.read_parquet('video_comments.parquet')
# Extract the comment_text column as a cuDF Series
#comments = cd_parquet['comment_text']

# Create the SentimentIntensityAnalyzer object
analyzer = SentimentIntensityAnalyzer()
# Define a cuDF function for sentiment analysis
def calculate_polarity(comment):
    return analyzer.polarity_scores(comment)['compound']

In [22]:
comments = cd_parquet['comment_text'].dropna()
start_time = time.time()

# Convert the comments to a list
comments_list = comments.to_pandas().tolist()

# Calculate sentiment polarity using CPU
polarity_scores = [calculate_polarity(comment) for comment in comments_list]
end_time = time.time()
print(f"Sentiment analysis took {end_time - start_time} seconds")


# Transfer the polarity scores back to the host (CPU)
#polarity_scores_host = cp.asnumpy(polarity_scores)



Sentiment analysis took 720.6410095691681 seconds


In [7]:
cd_parquet.memory_usage(index=False)

video_id        110674294
comment_text    655720034
likes            59035072
replies          59035072
dtype: int64

In [2]:
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# GPU
import cupy as cp
import cudf as cd
import numba
from multiprocessing import Pool



%time cd_parquet = cd.read_parquet('video_comments.parquet')

# Create the SentimentIntensityAnalyzer object
analyzer = SentimentIntensityAnalyzer()
# Define a cuDF function for sentiment analysis
def calculate_polarity(comment):
    return analyzer.polarity_scores(comment)['compound']

comments = cd_parquet['comment_text'].dropna()
start_time = time.time()

# Convert the comments to a list
comments_list = comments.to_pandas().tolist()

# Calculate sentiment polarity using multiprocessing
with Pool() as pool:
    %time polarity_scores = pool.map(calculate_polarity, comments_list)
end_time = time.time()
print(f"Sentiment analysis took {end_time - start_time} seconds")

CPU times: user 393 ms, sys: 93.3 ms, total: 486 ms
Wall time: 479 ms
CPU times: user 3.36 s, sys: 1.18 s, total: 4.54 s
Wall time: 2min 3s
Sentiment analysis took 126.19851636886597 seconds
